In [ ]:
import couchdb
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns

from dotenv import load_dotenv
from IPython.display import display

### 1. Verbindung zur existierenden Datenbank

In [ ]:
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984"  
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try: 
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich verbunden mit CouchDB unter {COUCHDB_URL}")
    
    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank {DB_NAME} erfolgreich ausgewählt")
    else:
        print(f"Datenbank {DB_NAME} nicht gefunden!")
        raise LookupError(f"Datenbank {DB_NAME} nicht gefunden!")

except Exception as e:
    print(f"Fehler beim Verbinden mit CouchDB: {e}")
    raise e

### Hilfsfunktion zum Erstellen/Aktualisieren von Design-Dokumenten

In [ ]:
def sync_design_doc(db_handle, design_doc_dict):
    if not db_handle:
        print("sync_design_doc: Keine Datenbankverbindung.")
        return False
        
    doc_id = design_doc_dict['_id']
    try:
        existing_doc = db_handle.get(doc_id)
        if existing_doc:
            if existing_doc.get('views') != design_doc_dict.get('views') or \
               existing_doc.get('language') != design_doc_dict.get('language'):
                design_doc_dict['_rev'] = existing_doc['_rev'] 
                db_handle.save(design_doc_dict)
                print(f"Design-Dokument '{doc_id}' aktualisiert.")
                return True
            else:
                print(f"Design-Dokument '{doc_id}' ist bereits aktuell.")
                return False
        else:
            # Design-Dokument existiert nicht, neu erstellen
            db_handle.save(design_doc_dict)
            print(f"Design-Dokument '{doc_id}' erstellt.")
            return True
    except Exception as e:
        print(f"Fehler beim Synchronisieren des Design-Dokuments '{doc_id}': {e}")
        return False

## Datenauswertung
### CIA World Factbook Analyse
* **Datenbasis**: CIA World Factbook mit globalen Länderinformationen
* **Ziel**: Identifikation interessanter Muster und Zusammenhänge zwischen verschiedenen Ländermerkmalen
* **Methodik**: Kombination von CouchDB-Abfragen mit MapReduce, Mango Queries und Python-basierter Visualisierung

## 1. Analyse: Wirtschaftliche Dynamik und Ungleichheit 

### Fragestellung
* **Kernfrage**: Welche Länder verzeichnen ein hohes Wirtschaftswachstum bei gleichzeitig hoher Einkommensungleichheit?
* **Wachstumsindikator**: Reales BIP-Wachstum > 4%
* **Ungleichheitsindikator**: Gini-Koeffizient > 40

### Ziele der Analyse
* Identifikation von Ländern mit problematischer Wohlstandsverteilung
* Aufzeigen von Regionen, in denen wirtschaftliches Wachstum nicht breit verteilt ist

### 1.1 CouchDB-Technik: Map-Reduce View

* **Design-Dokument**: `_design/economic_analysis`
* **View-Name**: `countries_by_geografische_region_filtered`

#### Technische Umsetzung:
* **Map-Funktion**: 
  - Filtert Länder nach GDP/Gini-Schwellenwerten
  - Extrahiert numerische Werte aus Text-/Zahlenformaten
  - Emittiert geografische Region als Schlüssel
  - Gruppiert Länderdaten pro Region

* **Reduce-Funktion**:
  - Aggregiert alle Länderinformationen einer Region
  - Ermöglicht Gruppierung und Zusammenfassung auf Serverseite

In [ ]:
wachstum_schwellenwert_for_view = 4.0
gini_schwellenwert_for_view = 40.0

if db:
    try:
        design_doc = db.get('_design/economic_analysis')
        if design_doc is None:
            design_doc = {'_id': '_design/economic_analysis', 'language': 'javascript', 'views': {}}
    except Exception:
        design_doc = {'_id': '_design/economic_analysis', 'language': 'javascript', 'views': {}}

    # Map-Funktion zur Filterung
    map_function_geografische_region_agg = f"""
    function(doc) {{
        var WACHSTUM_SCHWELLE = {wachstum_schwellenwert_for_view};
        var GINI_SCHWELLE = {gini_schwellenwert_for_view};
        if (doc.type === 'country' &&
            doc["Economy: Real GDP growth rate"] &&
            doc["Economy: Gini Index coefficient - distribution of family income"] &&
            doc["Government: Country name - conventional short form"] &&
            doc["Geography: Map references"]) {{
            var gdp_growth_str = doc["Economy: Real GDP growth rate"];
            var gini_str = doc["Economy: Gini Index coefficient - distribution of family income"];
            var country_name = doc["Government: Country name - conventional short form"];
            var geografische_region = doc["Geography: Map references"];
            var parsed_gdp_growth = null;
            var parsed_gini = null;
            
            if (typeof gdp_growth_str === 'string') {{
                var match_growth = gdp_growth_str.match(/(-?\\d+\\.?\\d*|-?\\d*\\.\\d+)/);
                if (match_growth && match_growth[0]) {{
                    parsed_gdp_growth = parseFloat(match_growth[0]);
                }}
            }} else if (typeof gdp_growth_str === 'number') 
            {{ parsed_gdp_growth = gdp_growth_str; }}
            if (typeof gini_str === 'string') {{
                var match_gini = gini_str.match(/(\\d+\\.?\\d*|\\d*\\.\\d+)/);
                if (match_gini && match_gini[0]) {{
                    parsed_gini = parseFloat(match_gini[0]);
                }}
            }} else if (typeof gini_str === 'number') 
            {{ parsed_gini = gini_str; }}
            
            // Filterung nach Kriterien
            if (parsed_gdp_growth > WACHSTUM_SCHWELLE && parsed_gini > GINI_SCHWELLE) {{
                // Emit der geografischen Region mit den gefilterten Werten
                emit(geografische_region, {{ "country": country_name, "gdp_growth": parsed_gdp_growth, "gini": parsed_gini }});         
            }}
        }}
    }}
    """

    #* Reduce-Funktion zur Aggregation pro geografische Region
    reduce_function_geografische_region_agg = f"""
    function(keys, values, rereduce) {{
        // rereduce -> aggregiert (sammelt) alle übergebenen values in einem gemeinsamen Array.
        var all_country_data = [];
        if (rereduce) {{
            for (var i = 0; i < values.length; i++) {{
                all_country_data = all_country_data.concat(values[i]);
            }}
        }} else {{
            all_country_data = values;
        }}
        return all_country_data;
    }}
    """
    
    # --- Die View ---
    if 'views' not in design_doc:
        design_doc['views'] = {}

    design_doc['views']['countries_by_geografische_region_filtered'] = {
        'map': map_function_geografische_region_agg,
        'reduce': reduce_function_geografische_region_agg
    }
    
    # Design-Dokument in CouchDB aktualisieren
    sync_design_doc(db, design_doc)
else:
    print("Keine Datenbankverbindung ('db'), Design-Dokument konnte nicht aktualisiert werden.")

### 1.2 **Abfrage der View aus Python:**
- Python fragt die View mit group=True ab und erhält direkt die aggregierten Listen.


In [ ]:
print(f"--- Aggregation nach geografischer Region via 'countries_by_geografische_region_filtered' ---")
print(f"Filterkriterien in der View: Wachstum > {wachstum_schwellenwert_for_view}% UND Gini Index > {gini_schwellenwert_for_view}")

try:
    view_results = db.view('economic_analysis/countries_by_geografische_region_filtered', group=True)
    # Flache Liste aller Länder mit Region, Land, Wachstum, Gini
    all_countries = [
        {
            'Geografische Region': row.key,
            'Land': country.get('country'),
            'Wachstumsrate (%) > 4%': country.get('gdp_growth'),
            'Gini-Index > 40': country.get('gini')
        }
        for row in view_results
        for country in (row.value if isinstance(row.value, list) else [])
    ]
    df_countries = pd.DataFrame(all_countries)
    if not df_countries.empty:
        print(f"\n{df_countries['Land'].nunique()} Länder erfüllen die Kriterien.")
        print("\n--- Gesamttabelle der gefilterten Länder ---")
        display(df_countries[['Land', 'Wachstumsrate (%) > 4%', 'Gini-Index > 40']].sort_values(by='Wachstumsrate (%) > 4%', ascending=False))
    else:
        print("\nKeine Länder im Profil gefunden. Es wird keine Tabelle oder Grafik für Analyse 1 erstellt.")
except Exception as e:
    print(f"Fehler beim Abfragen der View: {e}")

### 1.3 Aggregation der Ergebnisse

In [ ]:
df_summary = df_countries.groupby('Geografische Region').agg(
    Laender=('Land', lambda x: ', '.join(sorted(x))),
    Anzahl_Laender=('Land', 'count'),
    Durchschnitt_Wachstum=('Wachstumsrate (%) > 4%', 'mean'),
    Durchschnitt_Gini=('Gini-Index > 40', 'mean'),
    ).reset_index()

print(f"\n{df_countries['Geografische Region'].nunique()} geografische Regionen erfüllen die Kriterien.")
df_summary.round(2)

### 1.4 Visualisierung der Ergebnisse

In [ ]:
df_geografische_region_summary_plot = df_summary.sort_values(by='Anzahl_Laender', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(data=df_geografische_region_summary_plot,
            x='Anzahl_Laender', 
            y='Geografische Region', 
            palette='viridis', 
            hue='Geografische Region', 
            dodge=False)
plt.title(f'Anzahl Länder pro geografischer Region (Wachstum > {wachstum_schwellenwert_for_view}% & Gini > {gini_schwellenwert_for_view})', fontsize=14)
plt.xlabel('Anzahl Länder im Profil', fontsize=12)
plt.ylabel('Geografische Region', fontsize=12)
plt.tight_layout()
plt.show()

## 2. Analyse: Bildungsrendite-Paradoxon

### Konzept und Fragestellung
* **Definition: Bildungsrendite-Paradoxon**: Länder mit hohen Bildungswerten (hohe Schullebenserwartung), aber schlechten Beschäftigungsperspektiven (hohe Jugendarbeitslosigkeit) und hohem Abwanderung (negative Netto-Migrationsrate) -> ein widersprüchliches Muster.
* **Kernfrage**: Welche Länder zeigen ein Profil hoher Bildungswerte bei gleichzeitig Anzeichen für hohen Abwanderungsdruck und schlechten Beschäftigungsperspektiven?
* **Relevanz**: Identifikation von Regionen, die trotz hoher Bildungsniveaus von Talentabwanderung betroffen sind.

### Profil-Kategorisierung
* **Paradox-Profil**:
  - Hohe Schullebenserwartung (≥ 15 Jahre)
  - Negative Migration (< -1,0)
  - Hohe Jugendarbeitslosigkeit (≥ 13%)
* **Positiv-Profil**:
  - Hohe Schullebenserwartung (≥ 15 Jahre)
  - Positive Migration (> 0)
  - Niedrige Jugendarbeitslosigkeit (≤ 7%)

### Technischer Ansatz
* **Datenbankseite**:
  - Mango Queries mit flexible Abfragen
  - Map-Reduce View für effiziente Kategorisierung mit dem serverseitigen Parsen 
* **Clientseite**:
  - Filterung nach definierten Kriterien von CouchDB
  - Visuelle Darstellung der identifizierten Länderprofile

### 2.1 Serverseitige Mango Query

* **Anwendungsfall**:
  - Ad-hoc Datenexploration ohne View-Definition
  - Flexibler Zugriff auf alle Länder mit vollständigen Bildungs- und Migrationsdaten

In [ ]:
# Schwellenwerte für die Kategorisierung
SCHWELLE_SCHULLEBEN_KURZ = 8.0
SCHWELLE_SCHULLEBEN_LANG = 15.0
SCHWELLE_MIGRATION_NEGATIV = -1.0
SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX = 0.0
SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG = 7.0
SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH = 13.0

In [ ]:
mango_query = {
  "selector": {
    "type": "country",
    "People and Society: School life expectancy (primary to tertiary education) - total": {
      "$exists": True,
      "$ne": None
    },
    "People and Society: Net migration rate": {
      "$exists": True, 
      "$ne": None
    },
    "Economy: Youth unemployment rate (ages 15-24) - total": {
      "$exists": True,
      "$ne": None
    },
    "Geography: Map references": {
      "$exists": True,
      "$ne": None
    }
  },
  "fields": [
    "Government: Country name - conventional short form",
    "People and Society: School life expectancy (primary to tertiary education) - total",
    "People and Society: Net migration rate",
    "Economy: Youth unemployment rate (ages 15-24) - total",
  ],
  "limit": 1000
}
data = db.find(mango_query)
df = pd.DataFrame(data)

# Spalten umbenennen
df = df.rename(columns={
    "Government: Country name - conventional short form": "Country",
    "People and Society: School life expectancy (primary to tertiary education) - total": "School Life Expectancy",
    "People and Society: Net migration rate": "Net Migration Rate",
    "Economy: Youth unemployment rate (ages 15-24) - total": "Youth Unemployment Rate",
})

# Parse-Funktion 
def parse_number(val):
    if val is None:
        return None
    if isinstance(val, (int, float)):
        return val
    if isinstance(val, str):
        import re
        match = re.search(r'(-?\d+\.?\d*|-?\d*\.?\d+)', val)
        if match:
            return float(match.group())
    return None

# Spalten als float parsen
df['Net Migration Rate'] = df['Net Migration Rate'].apply(parse_number)
df['School Life Expectancy'] = df['School Life Expectancy'].apply(parse_number)
df['Youth Unemployment Rate'] = df['Youth Unemployment Rate'].apply(parse_number)

# Nullwerte entfernen 
df = df.dropna(subset=['Net Migration Rate', 'School Life Expectancy', 'Youth Unemployment Rate'])


# Filter mit exakt den gleichen Schwellenwerten 
df_filtered = df[
    (df['School Life Expectancy'] >= 15.0) &
    (df['Net Migration Rate'] < -1.0) &
    (df['Youth Unemployment Rate'] >= 13.0)
]

# Ergebnis
result_df = df_filtered.rename(columns={
    "Country": "country",
    "School Life Expectancy": "Schullebensdauer (J)",
    "Net Migration Rate": "Migration (pro 1000)",
    "Youth Unemployment Rate": "Jugendarbeitslosigkeit (%)"
})
result_df = result_df[['country', 'Schullebensdauer (J)', 'Migration (pro 1000)', 'Jugendarbeitslosigkeit (%)']]
result_df = result_df.sort_values(by=['Migration (pro 1000)'], ascending=[True])

print("\n--- Schwellenwerte für Kategorisierung ---")
print(f"  Schullebenserwartung (Jahre): \n\tKurz < {SCHWELLE_SCHULLEBEN_KURZ}, Mittel {SCHWELLE_SCHULLEBEN_KURZ}-{SCHWELLE_SCHULLEBEN_LANG}, Lang >= {SCHWELLE_SCHULLEBEN_LANG}")
print(f"  Netto-Migrationsrate (pro 1000): \n\tNegativ < {SCHWELLE_MIGRATION_NEGATIV}, Ausgeglichen {SCHWELLE_MIGRATION_NEGATIV} bis {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}, Positiv > {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}")
print(f"  Jugendarbeitslosigkeit (%): \n\tNiedrig < {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}, Mittel {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}-{SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}, Hoch >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}")

result_df

### 2.1.1 **Grafische Darstellung:**

   -  Die Länder, die einem der identifizierten Profile entsprechen, werden in einem Streudiagramm (Scatter Plot) visualisiert.
   -  **Achsen:** Jugendarbeitslosigkeit vs. Netto-Migrationsrate.
   -  Elemente der Grafik sind ein  Titel, klare Achsenbeschriftungen, eine Legende und die Annotation einiger Länderpunkte zur besseren Lesbarkeit.

In [ ]:
plt.figure(figsize=(15, 10))
sns.scatterplot(
    data=result_df,
    x='Jugendarbeitslosigkeit (%)',
    y='Migration (pro 1000)',
    size='Migration (pro 1000)', 
    hue='Schullebensdauer (J)',
    sizes=(100, 700),
    alpha=0.7,
    legend='full'
)

for i, row in result_df.iterrows():
    x_offset = 0.3
    y_offset = 0.1
   
    plt.annotate(
        row['country'],  
        xy=(row['Jugendarbeitslosigkeit (%)'], row['Migration (pro 1000)']),
        xytext=(row['Jugendarbeitslosigkeit (%)'] + x_offset, row['Migration (pro 1000)'] + y_offset),
        fontsize=10,
        bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7)
    )

plt.title("Länderprofile zum Bildungsrendite-Paradoxon", fontsize=14, fontweight='bold')
plt.xlabel('Jugendarbeitslosigkeit (%)', fontsize=12)
plt.grid(True, linestyle=':', alpha=0.6)
plt.ylabel('Netto-Migrationsrate (pro 1000 Einwohner)', fontsize=12)

### 2.2 **CouchDB Design-Dokument für Bildungsindikatoren**
* **Design-Dokument**: `_design/education_analysis`
* **View**:
  - `countries_by_region_and_category`: 
    - Gruppiert Länder nach geografischer Region und Kategorie (paradox/positiv)
    - Identifiziert Bildungsrendite-Paradoxon-Länder und positive Vergleichsfälle
Bildungsindikatoren

* **Kategorisierungskriterien**:
  - Paradox-Profil:
    - Hohe Schullebenserwartung (≥ 15 Jahre)
    - Negative Migration (< -1,0)
    - Hohe Jugendarbeitslosigkeit (≥ 13%)
  - Positiv-Profil:
    - Hohe Schullebenserwartung (≥ 15 Jahre)
    - Positive Migration (> 0)
    - Niedrige Jugendarbeitslosigkeit (< 7%)

* **Technische Umsetzung**:
* **Map-Funktionen**:
    - Extrahiert relevante Bildungs- und Migrationsdaten aus Länderdokumenten
    - Parst numerische Werte aus verschiedenen Datenformaten
    - Kategorisiert Länder basierend auf den definierten Schwellenwerten
    - Emittiert Schlüsselpaare aus [Region, Kategorie] mit Länderdaten
* **Reduce-Funktionen**:
    - Aggregiert Länderdaten pro Region und Kategorie
    - Ermöglicht effizientes Gruppieren und Filtern auf Datenbankebene
    - Unterstützt group_level-Parameter für flexible Aggregationstiefe

In [ ]:
if db:    
    # 1. View für Aggregation nach geografischer Region und Schwellenwertfilterung
    map_function_paradox_profile = f"""
    function(doc) {{
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"] &&
            doc["Geography: Map references"]) {{

            var country_name = doc["Government: Country name - conventional short form"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];
            var geografische_region = doc["Geography: Map references"];
            
            function parseNum(strVal) {{
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {{
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }}
                return null; 
            }}

            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_school_expectancy !== null && num_migration !== null && num_youth_unemployment !== null && geografische_region) {{
                // Filterung UND Kategorisierung in einem Schritt
                var category = null;
                
                // Paradox-Profil: Hohe Bildung, negative Migration, hohe Jugendarbeitslosigkeit
                if (num_school_expectancy >= {SCHWELLE_SCHULLEBEN_LANG} && //15 Jahre oder mehr
                    num_migration < {SCHWELLE_MIGRATION_NEGATIV} && // -1.0 oder weniger
                    num_youth_unemployment >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}) {{ // 13% oder mehr
                    category = "paradox"; // Hohe Bildung, negative Migration, hohe Jugendarbeitslosigkeit
                }}
                // Alternative Profile 
                else if (num_school_expectancy >= {SCHWELLE_SCHULLEBEN_LANG} && // 15 Jahre oder mehr
                         num_migration > {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX} && // 0 oder mehr Positive Migration
                         num_youth_unemployment <= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}) {{ // 7% oder weniger
                    category = "positiv";  // Hohe Bildung, positive Migration, niedrige Jugendarbeitslosigkeit
                }}
                
                if (category) {{
                    emit([geografische_region, category], {{ 
                        country: country_name,
                        Schullebensdauer: num_school_expectancy,
                        Migration: num_migration,
                        Jugendarbeitslosigkeit: num_youth_unemployment
                    }});
                }}
            }}
        }}
    }}
    """

    # Maßgeschneiderte Aggregation: Reduce-Funktion für die Schlüsselstruktur
    reduce_function_geografische_region = """
    function(keys, values, rereduce) {
        var all_country_data = [];
        if (rereduce) {
            for (var i = 0; i < values.length; i++) {
                all_country_data = all_country_data.concat(values[i]); // Teil-Ergebnisse weiter rekursiv verdichten
            }
        } else {
            all_country_data = values; // Erste Aggregationsebene: Werte direkt vom Map übernehmen
        }
        return all_country_data;
    }
    """
    
    # Design-Dokument 
    design_doc_education_paradox = {
        '_id': '_design/education_analysis', 
        'language': 'javascript',
        'views': {
            'countries_by_region_and_category': {
                'map': map_function_paradox_profile,
                'reduce': reduce_function_geografische_region
            }
        }
    }    
    
    sync_design_doc(db, design_doc_education_paradox)   
else:
    print("Keine Datenbankverbindung.")

### 2.3 Abfrage vorkategorisierter Daten
- Die vordefinierten Paradoxon-Profile direkt aus der CouchDB-View
- Abfrage aller Profile über die View mit Kategorien-Gruppierung


In [ ]:
print("Abfrage kategorisierter Länderprofile mit CouchDB View...")
countries_by_category = {}

try:
    # group_level=2 ermöglicht Gruppierung nach Region UND Kategorie
    results = db.view('education_analysis/countries_by_region_and_category', group_level=2)
    
    # Daten nach Kategorien sammeln
    for row in results:
        region = row.key[0]  # Erste Komponente des Schlüssels
        category = row.key[1] # Zweite Komponente des Schlüssels
        
        if category not in countries_by_category:
            countries_by_category[category] = []
            
        for country_data in row.value:
            country_data['geografische_region'] = region
            countries_by_category[category].append(country_data)
    
    # Kategorien 
    print(f"\nGefundene Kategorien: {', '.join(countries_by_category.keys())}")
    
    # Paradox-Kategorie 
    if 'paradox' in countries_by_category:
        paradox_countries = countries_by_category['paradox']
        df_paradox = pd.DataFrame(paradox_countries)
        
        # Anzahl der Paradox-Länder
        print(f"\n{len(df_paradox)} Länder erfüllen das Paradox-Profil: Hohe Bildung, negative Migration, hohe Jugendarbeitslosigkeit")
        
        # Aggregation nach Region für Paradox-Länder
        df_paradox_summary = df_paradox.groupby('geografische_region').agg(
            Anzahl_Laender=('country', 'count'),
            Laender=('country', lambda x: ', '.join(sorted(x))),
            Durchschnitt_Schullebensdauer=('Schullebensdauer', 'mean'),
            Durchschnitt_Migration=('Migration', 'mean'),
            Durchschnitt_Jugendarbeitslosigkeit=('Jugendarbeitslosigkeit', 'mean')
        ).reset_index().round(2)
        
        print("\n--- Durchschnittswerte nach Region für Paradox-Länder ---")
        display(df_paradox_summary.sort_values(by='Anzahl_Laender', ascending=False))
    else:
        print("\nKeine Länder in der Paradox-Kategorie gefunden.")

except Exception as e:
    print(f"Fehler beim Abrufen der View-Daten: {e}")

### 2.4 Analyse der Positiv-Länder

#### Profil-Kriterien
* **Bildung**: Hohe Schullebenserwartung (≥ 15 Jahre)
* **Migration**: Positive Netto-Migrationsrate (> 0)
* **Arbeitsmarkt**: Niedrige Jugendarbeitslosigkeit (< 7%)

In [ ]:
print("Abfrage und Analyse der Positiv-Länder (hohe Bildung, positive Migration)")
try:
    # Bereits gesammelte Liste der Länder in der "positiv"-Kategorie
    if 'positiv' in countries_by_category:
        positiv_countries = countries_by_category['positiv']
        df_positiv = pd.DataFrame(positiv_countries)

        print(f"\n{len(df_positiv)} Länder erfüllen das positive Profil: \nHohe Bildung: (≥ {SCHWELLE_SCHULLEBEN_LANG} Jahre) \nPositive Migration: (> {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX} pro 1000 Einwohner) \nNiedrige Jugendarbeitslosigkeit: (≤ {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}%)")

        # Ländertabelle mit allen Werten
        print("\n--- Detaillierte Übersicht aller Länder im positiven Profil ---")
        df_details = df_positiv.sort_values(by=['Migration'], ascending=False)
        display(df_details[['country', 'geografische_region', 'Schullebensdauer', 'Migration', 'Jugendarbeitslosigkeit']])
    else:
        print("Keine Länder in der positiv-Kategorie gefunden.")    
except Exception as e:
    print(f"Fehler bei der Analyse der Positiv-Länder: {e}")